In [3]:
#Mineral needs for dairy cows following NRC 2001
name = input('What is your name? \n')
import math

What is your name? 
 Aaron


In [4]:
import csv
with open('info.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
    writer.writerow(['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain'])

In [12]:
#Enter basic information for the calculation
def WriteNewRow():
    cowID = input('Cow ID#: ')
    bw = input('Body weight (lb, use whole numbers): ')
    mw = float(bw)**0.75
    dim = input('Days lactating? If not a lactating cow, type "0".\n')
    dp = input('Days pregnant? If not pregnant, type "0".\n')
    breed = input('What is the breed? Type "holstein", "jersey", or "other".')
    milkprod = input('Milk yield per day (kg/day): \n')
    milkfat = input('Percent milk fat: \n')
    wg = input('Average net bodyweight gain per day (lb/day): \n')
    NewRow = cowID,bw,mw,dim,dp,breed,milkprod,milkfat,wg
    import csv
    from csv import writer
    with open('info.csv', 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(NewRow)
    import csv
    with open('info.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)

In [13]:
WriteNewRow()

Cow ID#:  1111
Body weight (lb, use whole numbers):  500
Days lactating? If not a lactating cow, type "0".
 3
Days pregnant? If not pregnant, type "0".
 200
What is the breed? Type "holstein", "jersey", or "other". holstein
Milk yield per day (kg/day): 
 50
Percent milk fat: 
 3
Average net bodyweight gain per day (lb/day): 
 .7


['Cow ID', 'Body weight', 'mw', 'Days in milking', 'Days of pregnancy', 'Breed', 'Milk production', 'Milk fat', 'Weight gain']
['1111', '500', '105.7371263440564', '3', '200', 'holstein', '50', '10', '.5']
['1111', '500', '105.7371263440564', '3', '200', 'holstein', '50', '3', '.7']


In [14]:
def AddCowsQ():
    AddCows = input('Add another cow? (Y/N) \n')
    if AddCows == "Y":
        WriteNewRow()
        AddCowsQ()
    else:
        print('Ready to calculate Ca and P needs!')

In [15]:
AddCowsQ()

Add another cow? (Y/N) 
 N


Ready to calculate Ca and P needs!


In [8]:
#Below we will append Ca and P needs for each cow in new columns using Pandas

In [59]:
import pandas as pd
import numpy as np
import math
columns = ['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain']
Cows = pd.read_csv("./info.csv", usecols=columns)

#Urine Ca
try:
    Cows['CaUrine'] = 0.08*(Cows['Body weight'] / 100)
except:
    Cows['CaUrine'] = 'error'

#Fecal Ca
try:
    if ((Cows['Days in milking']) > 0).all():
        Cows['CaFecal'] = 3.1*(Cows['Body weight'] / 100)
    else:
        Cows['CaFecal'] = 1.54*(Cows['Body weight'] / 100)
except:
    Cows['CaFecal'] = 'error'

#Fetal Ca
try:
    if ((Cows['Days of pregnancy']) > 189).all():
        Cows['CaFetal'] = 0.02456*np.exp((0.05581-(0.00007*(Cows['Days of pregnancy'])))*(Cows['Days of pregnancy']))-0.02456*np.exp((0.05581-(0.00007*((Cows['Days of pregnancy'])-1)))*((Cows['Days of pregnancy'])-1))
    else:
        Cows['CaFetal'] = 0
except:
    Cows['CaFetal'] = 'error'

#Lactation Ca
try:
    if (Cows['Breed']).any() == 'holstein':
        Cows['CaMilkProd'] = 1.22*(Cows['Milk production'])
    elif (Cows['Breed']).any() == 'jersey':
        Cows['CaMilkProd'] = 1.45*(Cows['Milk production'])
    else:
        Cows['CaMilkProd'] = 1.37*(Cows['Milk production'])
    #print('Lactation =', round(float(CaMilkProd), 2))
except:
    Cows['CaMilkProd'] = 'error'

#Growing Ca
try:
    if ((Cows['Weight gain']) > 0).all():
        Cows['CaGrowing'] = (9.83*((Cows['mw'])**0.22)*(Cows['Body weight'])**-0.22)*((Cows['Weight gain'])/0.96)
    else:
        Cows['CaGrowing'] = 0
    #print('growing =', round(float(CaGrowing), 2))
except:
    Cows['CaGrowing'] = 'error'
       
Cows['Total Ca Need'] = Cows['CaUrine'] + Cows['CaFecal'] + Cows['CaFetal'] + Cows['CaMilkProd'] + Cows['CaGrowing']

Cows

,Cow ID,Body weight,mw,Days in milking,Days of pregnancy,Breed,Milk production,Milk fat,Weight gain,CaUrine,CaFecal,CaFetal,CaMilkProd,CaGrowing,Total Ca Need
0,1111,500,105.737126,3,200,holstein,50,10,0.5,0.4,15.5,2.891023,61.0,3.63755,83.428573
1,1111,500,105.737126,3,200,holstein,50,3,0.7,0.4,15.5,2.891023,61.0,5.09257,84.883593


In [19]:
#Function to calculate the calcium need
def calcium():
    #Needs of Ca for maintenance
    #Urinary
    try:
        Cows['CaUrine'] = 0.08*(Cows['Body weight'] / 100)
        #print('urinary =', round(float(CaUrine)), 2)
    except:
        Cows['CaUrine'] = 'error'
        
   #fecal
    try:
        if int(Cows['Days in milking']) > 0:
            Cows['CaFecal'] = 3.1*(Cows['Body weight'] / 100)
        else:
            Cows['CaFecal'] = 1.54*(Cows['Body weight'] / 100)
        #print('fecal =', round(float(CaFecal),2))
    except:
        Cows['CaFecal'] = 'error'
        
    #Pregnancy
    try:
        if float(Cows['Days of pregnancy']) > 189:
            Cows['CaFetal'] = 0.02456*math.exp((0.05581-(0.00007*float(Cows['Days of pregnancy'])))*float(Cows['Days of pregnancy']))-0.02456*math.exp((0.05581-(0.00007*(float(Cows['Days of pregnancy'])-1)))*(float(Cows['Days of pregnancy'])-1))
        else:
            Cows['CaFetal'] = 0
        #print('Fetal =', round(float(CaFetal), 2))
    except:
        Cows['CaFetal'] = 'error'
        
    #Lactation
    try:
        if str(Cows['Breed']) == 'holstein':
            Cows['CaMilkProd'] = 1.22*float(Cows['Milk production'])
        elif str(Cows['Breed']) == 'jersey':
            Cows['CaMilkProd'] = 1.45*float(Cows['Milk production'])
        else:
            Cows['CaMilkProd'] = 1.37*float(Cows['Milk production'])
        #print('Lactation =', round(float(CaMilkProd), 2))
    except:
        Cows['CaMilkProd'] = 'error'

    #Growing
    try:
        if float(Cows['Weight gain']) > 0:
            Cows['CaGrowing'] = (9.83*(float(Cows['mw'])**0.22)*(float(Cows['Body weight'])**-0.22))*(float(Cows['Weight gain'])/0.96)
        else:
            Cows['CaGrowing'] = 0
        #print('growing =', round(float(CaGrowing), 2))
    except:
        Cows['CaGrowing'] = 'error'
        
    Cows['Total Ca Need'] = Cows['CaFecal'] + Cows['CaUrine'] + Cows['CaFetal'] + Cows['CaMilkProd'] + Cows['CaGrowing']


In [59]:
#Function to calculate the calcium need
def calcium():
    #Needs of Ca for maintenance
    #Urinary
    try:
        Cows['CaUrine'] = 0.08*(Cows['Body weight'] / 100)
        #print('urinary =', round(float(CaUrine)), 2)
    except:
        Cows['CaUrine'] = 'error'
        
   #fecal
    try:
        if int(Cows['Days in milking']) > 0:
            Cows['CaFecal'] = 3.1*(Cows['Body weight'] / 100)
        else:
            Cows['CaFecal'] = 1.54*(Cows['Body weight'] / 100)
        #print('fecal =', round(float(CaFecal),2))
    except:
        Cows['CaFecal'] = 'error'
        
    #Pregnancy
    try:
        if float(Cows['Days of pregnancy']) > 189:
            Cows['CaFetal'] = 0.02456*math.exp((0.05581-(0.00007*float(Cows['Days of pregnancy'])))*float(Cows['Days of pregnancy']))-0.02456*math.exp((0.05581-(0.00007*(float(Cows['Days of pregnancy'])-1)))*(float(Cows['Days of pregnancy'])-1))
        else:
            Cows['CaFetal'] = 0
        #print('Fetal =', round(float(CaFetal), 2))
    except:
        Cows['CaFetal'] = 'error'
        
    #Lactation
    try:
        if str(Cows['Breed']) == 'holstein':
            Cows['CaMilkProd'] = 1.22*float(Cows['Milk production'])
        elif str(Cows['Breed']) == 'jersey':
            Cows['CaMilkProd'] = 1.45*float(Cows['Milk production'])
        else:
            Cows['CaMilkProd'] = 1.37*float(Cows['Milk production'])
        #print('Lactation =', round(float(CaMilkProd), 2))
    except:
        Cows['CaMilkProd'] = 'error'

    #Growing
    try:
        if float(Cows['Weight gain']) > 0:
            Cows['CaGrowing'] = (9.83*(float(Cows['mw'])**0.22)*(float(Cows['Body weight'])**-0.22))*(float(Cows['Weight gain'])/0.96)
        else:
            Cows['CaGrowing'] = 0
        #print('growing =', round(float(CaGrowing), 2))
    except:
        Cows['CaGrowing'] = 'error'
        
    Cows['Total Ca Need'] = Cows['CaFecal'] + Cows['CaUrine'] + Cows['CaFetal'] + Cows['CaMilkProd'] + Cows['CaGrowing']


In [ ]:
print('total Ca need =',round(float(CaNeed), 2))

In [ ]:
#Function to calculate the phosphorus need
def phosphorus():
    #Needs of Ca for maintenance
    #Urinary
    try:
        PUrine = 0.002*float(bw)
        #print('urinary =', round(float(PUrine), 2))
    except:
        print('Something on urine calculations is wrong')

    #fecal
    try:
        FCM = (0.4*float(milkprod))+(15*(float(milkfat)/100)*float(milkprod))
        DMI = ((0.372*float(FCM))+(0.0968*float(mw)))*(1-math.exp(-0.192*((float(dim)/7)+3.67)))
        PFecal = float(DMI)
        #print('fecal =', round(float(PFecal),2))
    except:
        print('Something on fecal calculations is wrong')
        
    #Pregnancy
    try:
        if float(dp) > 189:
            PFetal = 0.02743*math.exp((0.05527-(0.000075*float(dp)))*float(dp))-0.02743*math.exp((0.05527-(0.000075*(float(dp)-1)))*(float(dp)-1))
        else:
            PFetal = 0
        #print('Fetal =', round(float(PFetal), 2))
    except:
        print('Something on pregnancy calculations is wrong')
        
    #Lactation
    try:
        if int(dim) > 0:
            PMilkProd = 0.9*float(milkprod)
        else:
            PMilkProd = 0
        #print('Lactation =', round(float(PMilkProd), 2))
    except:
        print('Something on lactation calculations is wrong')
        
    #Growing
    try:
        if float(wg) > 0:
            PGrowing = (1.2+(4.635*((float(mw))**0.22)*((float(bw))**-0.22)))*(float(wg)/0.96)
        else:
            PGrowing = 0
        #print('growing =', round(float(PGrowing), 2))
    except:
        print('Something on growing calculations is wrong')
    
    PNeed = PFecal + PUrine + PFetal+ PMilkProd + PGrowing
    print('total P need =',round(float(PNeed), 2))
        


In [ ]:
#Run function and print results
ca = calcium()
P = phosphorus()

In [ ]:
#Add the results in the file with the information to produce a report file
import csv
with open('report.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
    writer.writerow(['Cow ID', 'Body weight', 'Days in milking', 'Days of pregnancy', 'Breed', 'Milk production', 'Milk fat', 'Weight gain'])
    writer.writerow([cowID, bw, dim, dp, breed, milkprod, milkfat, wg])
    writer.writerow('')
    writer.writerow(['Nutrient', 'Need'])
    writer.writerow(['Ca', ca])
    writer.writerow(['P', P])

In [ ]:
#Show the final report
import csv
with open('report.csv', 'r') as reportfile:
    reader = csv.reader(reportfile)
    for row in reader:
        print(row)

In [ ]:
import openpyxl
wb = openpyxl.load_workbook('Cow_CaP.xlsx')
ws = wb.active
i = 0
cell_val = ''
# Finds which row is blank first
while cell_val != '':
    cell_val = ws['A' + i].value
    i += 1
# Modify Sheet, Starting With Row i
wb.save('Cow_CaP.xlsx')